In [1]:
import requests as rq

In [2]:
resp = rq.get('https://en.wikipedia.org/wiki/William_Feller')

In [3]:
resp.content

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-disabled vector-feature-page-tools-pinned-disabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>William Feller - Wikipedia</title>\n<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-disabled vector-feature-page-tools-pinned-disabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled";(function(){var cookie=document.cookie

In [4]:
from bs4 import BeautifulSoup

In [5]:
advisor = BeautifulSoup(resp.content).find(lambda tag: tag.name=='tr' and 'student' in tag.text.lower())

In [6]:
advisors = advisor.find_all(lambda tag: tag.name=='a' and ('student' not in tag.text.lower()))

In [7]:
for item in advisors:
  print((item.text, item['href']))

('Patrick Billingsley', '/wiki/Patrick_Billingsley')
('George Forsythe', '/wiki/George_Forsythe')
('Henry McKean', '/wiki/Henry_McKean')
('Lawrence Shepp', '/wiki/Lawrence_Shepp')
('Hale Trotter', '/wiki/Hale_Trotter')
('Benjamin Weiss', '/wiki/Benjamin_Weiss')
('David A. Freedman', '/wiki/David_A._Freedman')


In [52]:
def get_page_info(page_url):
  # concat the base url
  base_url = 'https://en.wikipedia.org'
  url = base_url + page_url
  # send a request to receive the content
  resp = rq.get(url)
  content = resp.content
  # parse the contents
  soup = BeautifulSoup(content)
  result = {}
  for entry in ['student', 'advisor']:
    result[entry] = []
    entry_block = soup.find('table', {'class', 'infobox'}).find(lambda tag: tag.name=='tr' and entry in tag.text.lower())
    entries = entry_block.find_all(lambda tag: tag.name=='a' and (entry not in tag.text.lower()))
    for item in entries:
      if len(item.text) > 6:
        result[entry].append((item['href'], item.text))
  return result

In [62]:
import networkx as nx

init = ('/wiki/William_Feller', 'William Feller')

queue = [init]
visited_nodes = set()

graph = nx.DiGraph()

while len(queue) > 0:
  
  page = queue.pop(0)

  visited_nodes.add(page[0])

  try:
    data = get_page_info(page[0])
  except:
    continue

  for advisor in data['advisor']:
    if advisor[0] not in visited_nodes:
      queue.append(advisor)
      visited_nodes.add(advisor[0])
      graph.add_edge(advisor[1], page[1])

  for student in data['student']:
    if student[0] not in visited_nodes:
      queue.append(student)
      visited_nodes.add(student[0])
      graph.add_edge(page[1], student[1])

In [67]:
nx.write_edgelist(graph, 'graph.csv', delimiter=',', data=False)

In [68]:
!head -n 10 graph.csv

Richard Courant,William Feller
Richard Courant,Leifur Ásgeirsson
Richard Courant,Herbert Busemann
Richard Courant,Kurt Friedrichs
Richard Courant,Harold Grad
Richard Courant,Fritz John
Richard Courant,Joseph Keller
Richard Courant,Edgar Krahn
Richard Courant,Martin Kruskal
Richard Courant,Anneli Lax
